In [24]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("lemmatized_messages.csv")

convo_user = df[['id', 'match_id', 'averageConversationLength', 'lemmatized_message']]

convo_user.to_csv('finale.csv', index = False)

In [15]:
import pandas as pd
import json

# Load the file
file_path = 'news.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Define a function to extract relevant data
def extract_relevant_data(conversations_str):
    try:
        conversations = json.loads(conversations_str)
        relevant_data = []
        for convo in conversations:
            match_id = convo.get('match_id', '')
            for msg in convo.get('messages', []):
                message = msg.get('message', '')
                relevant_data.append({'match_id': match_id, 'message': message})
        return relevant_data
    except json.JSONDecodeError:
        return []

# Apply the function
df['extracted_data'] = df['conversations'].apply(extract_relevant_data)

# Explode the dataframe
exploded_df = df.explode('extracted_data')

# Extract 'match_id' and 'message'
exploded_df['match_id'] = exploded_df['extracted_data'].apply(lambda x: x.get('match_id') if isinstance(x, dict) else None)
exploded_df['message'] = exploded_df['extracted_data'].apply(lambda x: x.get('message') if isinstance(x, dict) else None)

# Drop unnecessary columns
final_df = exploded_df.drop(columns=['conversations', 'extracted_data'])

# Save to a new CSV file
output_file_path = 'newss.csv'  # Replace with your desired output path
final_df.to_csv(output_file_path, index=False)


In [17]:
# Let's try reading and merging the files again after the code execution state reset.

import pandas as pd

# Load the files
file_path_1 = 'newss.csv'
file_path_2 = 'new.csv'

df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combining the files using 'id' as a reference
combined_df = pd.merge(df1, df2, on='id')
combined_df.to_csv('newsss.csv', index = False)

/var/folders/j_/xgvzbrt119v6lnhbyn1b46940000gn/T/ipykernel_26154/3426602523.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file_path_1)


In [22]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re

# Ensure you have the necessary NLTK data
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# Function to convert NLTK's part of speech tags to wordnet's format
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load the CSV file
df = pd.read_csv('finalclean_noemoji.csv')

# Function to lemmatize a sentence
def lemmatize_sentence(sentence):
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence)
    # Lemmatize each word with its correct part-of-speech tag
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words])

# Clean and lemmatize the 'message' column
df['lemmatized_message'] = df['message'].apply(lambda x: lemmatize_sentence(re.sub(r'\W+', ' ', x.lower())))

# Save the modified DataFrame to a new CSV file
df.to_csv('lemmatized_messages.csv', index=False)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yosephyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yosephyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/yosephyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
file_path = 'finale.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Create a Bag of Words model with a threshold of 3
vectorizer = CountVectorizer(min_df=5)
X = vectorizer.fit_transform(df['message'])

# Create a binary target variable: 1 if above average, 0 if below
average_length = df['averageConversationLength'].mean()
y = (df['averageConversationLength'] > average_length).astype(int)

# Split the dataset into training and development datasets
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.3, random_state=42)

# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg_train_accuracy = accuracy_score(y_train, log_reg.predict(X_train))
log_reg_dev_accuracy = accuracy_score(y_dev, log_reg.predict(X_dev))

# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_train_accuracy = accuracy_score(y_train, nb_model.predict(X_train))
nb_dev_accuracy = accuracy_score(y_dev, nb_model.predict(X_dev))

"""
# Multilayer Perceptron (MLP)
mlp_model = MLPClassifier(max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)
mlp_train_accuracy = accuracy_score(y_train, mlp_model.predict(X_train))
mlp_dev_accuracy = accuracy_score(y_dev, mlp_model.predict(X_dev))

# Random Forest

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_train_accuracy = accuracy_score(y_train, rf_model.predict(X_train))
rf_dev_accuracy = accuracy_score(y_dev, rf_model.predict(X_dev))"""

# Print out the accuracies for each model
print("Logistic Regression - Training Accuracy:", log_reg_train_accuracy, "Development Accuracy:", log_reg_dev_accuracy)
print("Naive Bayes - Training Accuracy:", nb_train_accuracy, "Development Accuracy:", nb_dev_accuracy)
#print("Multilayer Perceptron - Training Accuracy:", mlp_train_accuracy, "Development Accuracy:", mlp_dev_accuracy)
#print("Random Forest - Training Accuracy:", rf_train_accuracy, "Development Accuracy:", rf_dev_accuracy)


Logistic Regression - Training Accuracy: 0.7820147979510529 Development Accuracy: 0.7463986675069776
Naive Bayes - Training Accuracy: 0.7506825000241166 Development Accuracy: 0.7321508958314576


/Users/yosephyan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
